In [1]:
# For pvlib
import pvlib
import pandas as pd # 'as pd' to change alias from pandas to pd
import matplotlib.pyplot as plt
import pgeocode

# For demandlib
import datetime
from datetime import time as settime
import numpy as np
from demandlib import bdew
import demandlib.bdew as bdew
import demandlib.particular_profiles as profiles
from workalendar.europe import Germany

# Input
plz = 40599
anlage_groesse = 10 
strom_bedarf = 4000
waerme_bedarf = 20000


# Standort
nomi = pgeocode.Nominatim('de') 
a = nomi.query_postal_code(plz)
latitude = a['latitude']
longitude = a['longitude']

# Get hourly solar irradiation and modeled PV power output from PVGIS
data, meta, inputs = pvlib.iotools.get_pvgis_hourly(latitude, longitude, start=2016, end=2016, surface_tilt=35,
                                                    pvcalculation=True, peakpower=anlage_groesse, mountingplace='building', loss = 0)  

# Change Index to match demandlib index
year = 2016
data.set_index(pd.date_range(datetime.datetime(year, 1, 1, 0), periods= len(data), freq="h"), inplace=True)

## Stromprofil - demandlib ##
# Get holidays
cal = Germany()
holidays = dict(cal.holidays(year))

ann_el_demand_per_sector = {
    "h0_dyn": strom_bedarf,
}

# read standard load profiles
e_slp = bdew.ElecSlp(year, holidays=holidays)

# multiply given annual demand with timeseries
elec_demand = e_slp.get_profile(ann_el_demand_per_sector) # 15 Minuten Takt
df = elec_demand.resample("h").mean() # 1 Stunde Takt
df['pv'] = data['P'].div(1000)
df

c:\Users\DianaEspinosaLozano\anaconda3\envs\thesis\Lib\site-packages\demandlib\bdew\elec_slp.py:155: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[7 7 7 ... 6 6 6]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  new_df.update(merged_df)
c:\Users\DianaEspinosaLozano\anaconda3\envs\thesis\Lib\site-packages\demandlib\bdew\elec_slp.py:155: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0  0  0 ... 23 23 23]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  new_df.update(merged_df)
c:\Users\DianaEspinosaLozano\anaconda3\envs\thesis\Lib\site-packages\demandlib\bdew\elec_slp.py:155: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0 15 30 ... 15 30 45]' has dtype incompatible with int32, please expl

,h0_dyn,pv
2016-01-01 00:00:00,0.386959,0.0
2016-01-01 01:00:00,0.278753,0.0
2016-01-01 02:00:00,0.223167,0.0
2016-01-01 03:00:00,0.203622,0.0
2016-01-01 04:00:00,0.191801,0.0
...,...,...
2016-12-31 19:00:00,1.061405,0.0
2016-12-31 20:00:00,0.868713,0.0
2016-12-31 21:00:00,0.677457,0.0
2016-12-31 22:00:00,0.618953,0.0


In [ ]:
start_date = datetime.datetime(2016, 1, 1)

# Loop through each hour of the year
for i in range(df):
    # Calculate the current datetime based on the hour offset
    current_time = start_date + datetime.timedelta(hours=i)
    
    # Get the current day of the week (0 = Monday, 6 = Sunday)
    day_of_week = current_time.weekday()
    
    # Get the current month and day
    current_month = current_time.month
    current_day = current_time.day

    # You can now differentiate between months, weekdays, weekends, etc.
    # For example, checking if it's a weekend:
    if day_of_week >= 5:  # Saturday (5) or Sunday (6)
        is_weekend = True
    else:
        is_weekend = False

    # Perform the EV simulation logic here, using:
    # - current_time for the specific hour, day, and month
    # - day_of_week for weekday/weekend logic
    # - current_month and current_day for month-specific logic
    # Example:
    print(f"Hour {i}: {current_time} - {'Weekend' if is_weekend else 'Weekday'}")